In [ ]:
#https://www.programmableweb.com/category/all/web-development-frameworks?page=0

In [ ]:
!pip install requests-random-user-agent
!pip install pandas
!pip install bs4
!pip install lxml

     |████████████████████████████████| 51kB 4.9MB/s 


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36


In [ ]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_FRAME(url):

  df_tempSDK = pd.DataFrame()

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSDK = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSDK = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
       text = row.find_all('td')[0]      
       list_urlSDK.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0])) # Meta URL 


  df_tempSDK['Meta_Url'] = list_urlSDK

  return df_tempSDK


def download_FRAME(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_FRAME, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [ ]:
FRAME_urls=[]
for i in range(23): ## web pages?.. 22+1
    main_url = 'https://www.programmableweb.com/category/all/web-development-frameworks?page=' + str(i)
    FRAME_urls.append(main_url)

In [ ]:
#SDK_urls
df_FRAME = pd.DataFrame()
df_FRAME = download_FRAME(FRAME_urls, 5) # Num workers
#df_SDK.drop('Published', inplace=True, axis=1)
df_FRAME

,Description,Published,Meta_Url
Framework Name,,,
Spring Framework,"Spring is a powerful open source, loosely coup...",02.19.2021,https://www.programmableweb.com/framework/spri...
ComposerJS Framework,ComposerJS is a framework for building client ...,06.04.2020,https://www.programmableweb.com/framework/comp...
Hanami,"The Hanami Ruby Framework is a minimal, extrem...",03.05.2020,https://www.programmableweb.com/framework/hanami
Laravel,The Laravel PHP Framework in version 7 has bee...,NaN,https://www.programmableweb.com/framework/laravel
Kivy Python Framework,Kivy Python enables cross-platform OpenGL ES 2...,01.23.2020,https://www.programmableweb.com/framework/kivy...
...,...,...,...
Slim PHP Framework by Slim,Slim is a lightweight micro framework that dev...,04.28.2014,https://www.programmableweb.com/framework/slim...
Spring Social Core,Includes Spring Social's core service provider...,11.30.2011,https://www.programmableweb.com/framework/spri...
140dev Twitter Framework,The 140dev Twitter Framework is a free source ...,04.01.2011,https://www.programmableweb.com/framework/140d...


In [ ]:
# Creates new columns
df_FRAME['Languages'] = ""
df_FRAME['Provider'] = ""
df_FRAME['Asset URL'] = ""
df_FRAME['Repository'] = ""
df_FRAME['Terms Of Service'] = ""

In [ ]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    #print(df_source)
    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text
    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]
    
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    for lab in meta_specs.select("label"):   

      # Search for Languages
      if (lab.text.lower().find("languages") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Languages'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("provider") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Provider'][i] =   lab.find_next_sibling().text

      # Search for URL
      if (lab.text.lower().find("asset home") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Asset URL'][i] =   lab.find_next_sibling().text

      # Search for Repo
      if (lab.text.lower().find("repository") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Repository'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("terms of service") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Terms Of Service'][i] =   lab.find_next_sibling().text


    #print(df_source)

  return df_source


def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp

In [ ]:
df_FRAME_metaData = pd.DataFrame()
df_FRAME_metaData = download_meta_url(df_FRAME, 5)
df_FRAME_metaData

,Description,Published,Meta_Url,Languages,Provider,Asset URL,Repository,Terms Of Service
Framework Name,,,,,,,,
Spring Framework,"Spring is a powerful open source, loosely cou...",02.19.2021,https://www.programmableweb.com/framework/spri...,Java,VMWare,https://www.interviewbit.com/spring-interview-...,https://spring.io/quickstart,https://www.vmware.com/help/legal.html
ComposerJS Framework,ComposerJS is a framework for building client...,06.04.2020,https://www.programmableweb.com/framework/comp...,JavaScript,AcceleratXR,https://www.composerjs.io/about.html,https://www.composerjs.io/reference/server/ind...,
Hanami,"The Hanami Ruby Framework is a minimal, extre...",03.05.2020,https://www.programmableweb.com/framework/hanami,Ruby,,https://hanamirb.org/blog/2020/02/26/introduci...,,
Laravel,The Laravel PHP Framework in version 7 has be...,NaN,https://www.programmableweb.com/framework/laravel,PHP,Laravel,https://laravel.com/docs/7.x/releases,,
Kivy Python Framework,Kivy Python enables cross-platform OpenGL ES ...,01.23.2020,https://www.programmableweb.com/framework/kivy...,Python,Kivy,https://github.com/kivy/kivy,,
...,...,...,...,...,...,...,...,...
Slim PHP Framework by Slim,Slim is a lightweight micro framework that de...,04.28.2014,https://www.programmableweb.com/framework/slim...,PHP,,http://www.slimframework.com/,https://github.com/slimphp/Slim,
Spring Social Core,Includes Spring Social's core service provide...,11.30.2011,https://www.programmableweb.com/framework/spri...,,,http://static.springsource.org/spring-social/d...,,
140dev Twitter Framework,The 140dev Twitter Framework is a free source...,04.01.2011,https://www.programmableweb.com/framework/140d...,,,http://140dev.com/free-twitter-api-source-code...,,


In [ ]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_FRAME_metaData.reset_index(inplace=True)
df_FRAME_metaData

,Framework Name,Description,Published,Meta_Url,Languages,Provider,Asset URL,Repository,Terms Of Service
0,Spring Framework,"Spring is a powerful open source, loosely cou...",02.19.2021,https://www.programmableweb.com/framework/spri...,Java,VMWare,https://www.interviewbit.com/spring-interview-...,https://spring.io/quickstart,https://www.vmware.com/help/legal.html
1,ComposerJS Framework,ComposerJS is a framework for building client...,06.04.2020,https://www.programmableweb.com/framework/comp...,JavaScript,AcceleratXR,https://www.composerjs.io/about.html,https://www.composerjs.io/reference/server/ind...,
2,Hanami,"The Hanami Ruby Framework is a minimal, extre...",03.05.2020,https://www.programmableweb.com/framework/hanami,Ruby,,https://hanamirb.org/blog/2020/02/26/introduci...,,
3,Laravel,The Laravel PHP Framework in version 7 has be...,NaN,https://www.programmableweb.com/framework/laravel,PHP,Laravel,https://laravel.com/docs/7.x/releases,,
4,Kivy Python Framework,Kivy Python enables cross-platform OpenGL ES ...,01.23.2020,https://www.programmableweb.com/framework/kivy...,Python,Kivy,https://github.com/kivy/kivy,,
5,FastAPI Python Framework,The FastAPI Python Framework builds APIs. Wit...,05.10.2019,https://www.programmableweb.com/framework/fast...,Python,,https://fastapi.tiangolo.com/,https://github.com/tiangolo/fastapi,
6,Jose JavaScript Framework,Jose Framework means Javascript Object Signin...,05.02.2019,https://www.programmableweb.com/framework/jose...,JavaScript,,https://jose.readthedocs.io/en/latest/,,
7,Ravel Node.js Framework,The Ravel Node.js Framework assists in applic...,05.02.2019,https://www.programmableweb.com/framework/rave...,Node.js,,http://raveljs.github.io/docs/latest/,https://github.com/raveljs/ravel,
8,Google AMP JavaScript Framework,AMP is a JavaScript web component framework t...,05.02.2019,https://www.programmableweb.com/framework/goog...,JavaScript,Google,https://developers.google.com/amp,https://playground.amp.dev,https://github.com/ampproject/docs/blob/future...
9,Planaria JavaScript Framework,The Planaria Framework allows you to build a ...,03.25.2019,https://www.programmableweb.com/framework/plan...,JavaScript,Planaria,https://docs.planaria.network/#/intro,https://github.com/interplanaria/planaria,https://github.com/interplanaria/planaria/blob...


In [ ]:
from datetime import datetime
datetime = datetime.now()

## Export the original + Meta_Url
# To CSV (index True 0,1,2...)
df_FRAME_metaData.to_csv(r'/content/DataFrames/FRAME_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_FRAME_metaData.to_json(r'/content/DataFrames/FRAME_' + datetime.now().strftime('%d_%m_%Y') + '.json')